In [29]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

preprocessor_bert = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
encoder_bert = hub.KerasLayer(
    "https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-l-12-h-768-a-12/versions/4")

train_set = pd.read_csv('/content/compiled_processed_train_data.csv')
val_set = pd.read_csv('/content/compiled_processed_val_data.csv')

X_train = train_set['text']
X_val = val_set['text']
y_train = train_set.generated
y_val = val_set.generated

In [30]:
# Build the neural network
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_input')
encoder_inputs_bert = preprocessor_bert(text_input)
outputs_bert = encoder_bert(encoder_inputs_bert)

layer = tf.keras.layers.Dropout(0.2)(outputs_bert['pooled_output'])
layer = tf.keras.layers.Dense(1, activation='sigmoid')(layer)  # sentence embedding

model = tf.keras.Model(inputs=[text_input], outputs=[layer])

In [31]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text_input (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 keras_layer_8 (KerasLayer)  {'input_word_ids': (None,    0         ['text_input[0][0]']          
                             128),                                                                
                              'input_type_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                              

In [32]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall()])

In [33]:
model.fit(X_train,
          y_train, epochs=30,
          validation_data=(X_val, y_val),
          callbacks=[early_stopping])

Epoch 1/30
55/55 [==============================] - 42s 626ms/step - loss: 0.6757 - binary_accuracy: 0.6102 - precision_4: 0.4373 - recall_4: 0.2009 - val_loss: 0.5911 - val_binary_accuracy: 0.6445 - val_precision_4: 1.0000 - val_recall_4: 0.0373
Epoch 2/30
55/55 [==============================] - 31s 569ms/step - loss: 0.5847 - binary_accuracy: 0.6900 - precision_4: 0.6574 - recall_4: 0.3318 - val_loss: 0.5179 - val_binary_accuracy: 0.6789 - val_precision_4: 1.0000 - val_recall_4: 0.1304
Epoch 3/30
55/55 [==============================] - 26s 472ms/step - loss: 0.5032 - binary_accuracy: 0.7687 - precision_4: 0.7786 - recall_4: 0.5202 - val_loss: 0.4662 - val_binary_accuracy: 0.7110 - val_precision_4: 0.9730 - val_recall_4: 0.2236
Epoch 4/30
55/55 [==============================] - 26s 478ms/step - loss: 0.4561 - binary_accuracy: 0.7991 - precision_4: 0.8093 - recall_4: 0.5950 - val_loss: 0.4122 - val_binary_accuracy: 0.8096 - val_precision_4: 0.9643 - val_recall_4: 0.5031
Epoch 5/30
5

In [35]:
model.save('model_bert_untrainable_no_prompt_id.keras')

!pip install tensorflow tensorflow-text tensorflow_hub pandas